####This code wraps the C function "parametric2" for use in Python. This function solves CAPME problem using parametric simplex method
Primary Authors of C function: Haotian Pang, Han Liu and Robert Vanderbei     
Emails: <hpang@princeton.edu>, <hanliu@princeton.edu> and <rvdb@princeton.edu>  
Modified for covariate adjusted precision matrix estimation by: John Pura                                               
Email: <john.pura@duke.edu>                                                    

In [6]:
from __future__ import division
import os
import sys
import glob

In [7]:
%%file parametric2.h

int ratio_test(double *dy, int *idy,int ndy, double *y, double mu);
void parametric2(double *SigmaInput_xx, double *SigmaInput_xy, int *m1, double *mu_input, double *lambdamin, int *nlambda, int *maxnlambda, double *iicov);
void solver2(int m,int n,int nz,int *ia, int *ka, double *a,double *b, double *c);

Overwriting parametric2.h


In [8]:
%%file _parametric2.pyx

import numpy as np
cimport numpy as np

np.import_array()

# cdefine the signature of the c function
cdef extern from "parametric2.h":
    void parametric2(double *SigmaInput_xx, double *SigmaInput_xy, int *m1, double *mu_input, 
                    double *lambdamin, int *nlambda, int *maxnlambda, double *iicov);
    
def mainfunc(np.ndarray[double, ndim=2, mode="c"] SigmaInput_xx not None,
             np.ndarray[double, ndim=2, mode="c"] SigmaInput_xy not None,
             double lambdamin, 
             int nlambda):   

    #Dimensions    
    cdef int m1 = SigmaInput_xx.shape[1]
    
    #Define output
    cdef int maxnlambda = 0
    cdef np.ndarray mu_input = np.zeros((m1,nlambda), dtype = np.float64, order='C')
    cdef np.ndarray iicov = np.zeros((m1*m1,nlambda), dtype = np.float64, order='C')

    #Call external C function
    parametric2(<double*> np.PyArray_DATA(SigmaInput_xx),
               <double*> np.PyArray_DATA(SigmaInput_xy),
               &m1,
               <double*> np.PyArray_DATA(mu_input),
               <double*> &lambdamin,
               &nlambda,
               &maxnlambda,
               <double*> np.PyArray_DATA(iicov)
              )
    
    return (SigmaInput_xx,SigmaInput_xy,mu_input,maxnlambda,iicov)


Overwriting _parametric2.pyx


In [9]:
%%file setup.py

from distutils.core import setup, Extension
import numpy as np
import numpy.distutils.misc_util
from Cython.Distutils import build_ext

setup(
    name = 'parametric2',
    version = '1.0',
    description = 'C wrapper for parametric2 function',
    author = 'John Pura',
    author_email = 'john.pura@duke.edu',
    cmdclass={'build_ext': build_ext},
    ext_modules=[Extension("parametric2",
                           sources=["_parametric2.pyx", "parametric2.c",
                                    "lu.c","tree.c","linalg.c",
                                    "heap.c"],
                 include_dirs=[np.get_include(), ])]
)


Overwriting setup.py


In [10]:
!python setup.py build_ext --inplace

running build_ext
cythoning _parametric2.pyx to _parametric2.c
building 'parametric2' extension
C compiler: gcc -pthread -fno-strict-aliasing -g -O2 -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC

compile options: '-I/home/bitnami/anaconda/lib/python2.7/site-packages/numpy/core/include -I/home/bitnami/anaconda/include/python2.7 -c'
gcc: heap.c
gcc: linalg.c
gcc: _parametric2.c
In file included from /home/bitnami/anaconda/lib/python2.7/site-packages/numpy/core/include/numpy/ndarraytypes.h:1804:0,
                 from /home/bitnami/anaconda/lib/python2.7/site-packages/numpy/core/include/numpy/ndarrayobject.h:17,
                 from /home/bitnami/anaconda/lib/python2.7/site-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from _parametric2.c:232:
/home/bitnami/anaconda/lib/python2.7/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:15:2: warning: #warning "Using deprecated NumPy API, disable it by " "#defining NPY_NO_DEPRECATED_API NPY_1_7_API_